# Train Network for feature extraction

Feed MFCC values for each song to and encoder-decoder network.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import cPickle
import numpy as np

import theano
from keras.models import Sequential, model_from_yaml
from keras.layers.recurrent import LSTM
from keras.layers.core import Activation, Dense, Dropout, TimeDistributedDense, RepeatVector
from keras.preprocessing import sequence
import yaml
import os
import time

## Read data
Pad items with max length of 150

X.shape = (N, 150, 20)

In [ ]:
# Read data
config = yaml.load(open("../config.yaml").read())
beets_config = yaml.load(open(config["beets_config"]).read())
seq_features = cPickle.load(open(beets_config["blackbird"]["seq_features"], "rb"))
weights_file = beets_config["blackbird"]["lstm"]["weights"]
arch_file = beets_config["blackbird"]["lstm"]["arch"]

maxlen = 150

X = np.empty((len(seq_features), maxlen, 20))

for idx, key in enumerate(seq_features):
    X[idx, :, :] = sequence.pad_sequences(seq_features[key], maxlen=maxlen, dtype="float32").T

## Train
Reconstruct sequences from a dense vector of size 20

In [ ]:
# Create model

model = Sequential()
model.add(LSTM(64, return_sequences=False, input_shape=(maxlen, 20)))
model.add(Dropout(0.5))
model.add(Dense(20))
model.add(Activation("tanh"))
model.add(RepeatVector(maxlen))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributedDense(20))

model.compile(loss="mse", optimizer="adam")

In [ ]:
# Train
history = model.fit(X, X, batch_size=128, nb_epoch=250, validation_split=0.2, verbose=2)

In [ ]:
# Dump history
cPickle.dump(history.history, open("train_history.pickle", "wb"))

In [ ]:
# Save architecture and weights
if os.path.isfile(weights_file):
    os.rename(weights_file, weights_file + ".backup." + str(time.time()))
if os.path.isfile(arch_file):
    os.rename(arch_file, arch_file + ".backup." + str(time.time()))
    
# Save things
open(arch_file, "w").write(model.to_yaml())
model.save_weights(weights_file)

## Load previous model

In [ ]:
# Load model
model = model_from_yaml(open(arch_file).read())
model.load_weights(weights_file)

In [ ]:
# Function to predict output
predict = theano.function([model.layers[0].input],
                          model.layers[3].get_output(train=False),
                          allow_input_downcast=True)